# 4.2 模型参数的访问、初始化和共享

In [1]:
import torch
from torch import nn
from torch.nn import init

print(torch.__version__)

1.9.0


In [3]:
net = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3, 1))  # pytorch已进行默认初始化

print(net)
X = torch.rand(2, 4)
Y = net(X).sum()
Y

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


tensor(-0.3245, grad_fn=<SumBackward0>)

In [6]:
net.named_parameters, net.parameters

(<bound method Module.named_parameters of Sequential(
   (0): Linear(in_features=4, out_features=3, bias=True)
   (1): ReLU()
   (2): Linear(in_features=3, out_features=1, bias=True)
 )>,
 <bound method Module.parameters of Sequential(
   (0): Linear(in_features=4, out_features=3, bias=True)
   (1): ReLU()
   (2): Linear(in_features=3, out_features=1, bias=True)
 )>)

## 4.2.1 访问模型参数

In [8]:
print(type(net.named_parameters()))
for name, param in net.named_parameters():
    print(name, param.size())

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [9]:
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


In [10]:
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20, 20))
        self.weight2 = torch.rand(20, 20)
    def forward(self, x):
        pass
    
n = MyModel()
for name, param in n.named_parameters():
    print(name)

weight1


In [11]:
weight_0 = list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad)
Y.backward()
print(weight_0.grad)

tensor([[ 0.2417, -0.1945,  0.0691,  0.2496],
        [-0.3902, -0.3560, -0.0775,  0.0052],
        [-0.1103, -0.1662, -0.4498,  0.3994]])
None
tensor([[-0.7184, -0.6142, -0.7561, -0.4035],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000]])


## 4.2.2 初始化模型参数

In [18]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        print(name, param.data)
#         print(net.parameters)

0.weight tensor([[-0.0118,  0.0092, -0.0034,  0.0060],
        [ 0.0099, -0.0010,  0.0134, -0.0020],
        [-0.0052,  0.0025, -0.0053, -0.0070]])
2.weight tensor([[ 0.0050,  0.0254, -0.0162]])


In [19]:
for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


## 4.2.3 自定义初始化方法

In [38]:
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10, 10)
        print(f"tensor - {tensor}")
#         print(f"|tensor|>5 - {tensor.abs() >= 5}")
#         print(f"|tensor|>5 - {(tensor.abs() >= 5).float()}")
        tensor *= (tensor.abs() >= 5).float()

In [39]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name, param.data)

tensor - Parameter containing:
tensor([[ 8.4747, -7.8043, -2.7209, -8.4373],
        [ 9.8066, -6.6392,  5.9179, -4.2216],
        [-5.7650,  8.1102,  8.1013, -6.5477]], requires_grad=True)
0.weight tensor([[ 8.4747, -7.8043, -0.0000, -8.4373],
        [ 9.8066, -6.6392,  5.9179, -0.0000],
        [-5.7650,  8.1102,  8.1013, -6.5477]])
tensor - Parameter containing:
tensor([[-2.9854,  8.8126, -8.8309]], requires_grad=True)
2.weight tensor([[-0.0000,  8.8126, -8.8309]])


In [41]:
for name, param in net.named_parameters():
    if 'bias' in name:
        param.data += 1
        print(name, param.data)

0.bias tensor([2., 2., 2.])
2.bias tensor([2.])


## 4.2.4 共享模型参数

In [42]:
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear) 
print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [43]:
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))

True
True


In [44]:
x = torch.ones(1, 1)
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad)

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])
